## Recommender Systems

Students: Irene Cantero (U151206) / Jian Chen (U150279)

This notebook contains the 4 algorithms requested in the project sentence + 1 algorithm chose by us.

Content:

- Alternate Least Squares (ALS)
- Adamic-Adar
- Personalized PageRank
- Node2Vec
- Doc2Vec (chose by us)

In [1]:
from search_engine.search_engine import SearchEngine
import networkx as nx
from networkx import Graph
from sklearn.model_selection import train_test_split
import implicit
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import igraph
import warnings
import csv
import argparse
import numpy as np
import networkx as nx
import node2vec
from gensim.models import Word2Vec
warnings.filterwarnings('ignore')

In [2]:
search_engine = SearchEngine()

Collection time: 0.0


## Graph Creation

In [3]:
#Create a graph where the vertices are formed by the users that retweet (users u) and the retweeted users (users v)
#And the edge is the connection of users u to users v
graph=igraph.Graph()
for tweet in search_engine.tweets.iterrows():
    if str(tweet[1]['retweeted_status'])!='nan':
        u=tweet[1]['user']['screen_name']
        v=tweet[1]['retweeted_status']['user']['screen_name']
        graph.add_vertices(u)
        graph.add_vertices(v)
        graph.add_edges([(u,v)])

In [4]:
#SELECT USER:
# If you change the user id, the recommendation of all 4 algorithms will try to satisfy that user.
user_id=0
user_name=graph.vs[user_id]['name']

 ----------------------------------------------------------------------------------------------------------------------------

## Data Separation

In [10]:
def nodes_at_distance_2(graph: igraph.Graph, test_idxs: list) -> set:
    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        if n1.index in test_idxs:
            # all the nodes at distance 1
            nodes_at_most_distant_1 = set(graph.neighborhood(n1, order=1))
            # all the nodes at distance 1 and distance 2
            nodes_at_most_distant_2 = set(graph.neighborhood(n1, order=2))
            # only the nodes at distance 2
            only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1

            # check if empty set
            if len(only_nodes_at_distance_2) > 0:
                for n2 in only_nodes_at_distance_2:
                    # since n1 is an igraph vertex object, we need to extract the id
                    n1_index = n1.index
                    all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

In [11]:
# fraction of edges to select as test-set
p = 0.2
# graphsize
N = len(graph.es)
# idxs of all the edges
all_idxs = range(N)
# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

In [12]:
ground_truth = set()

for idx, one_edge in enumerate(graph.es):
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))

In [13]:
all_potential_recommendations = nodes_at_distance_2(graph, list(test_idxs))
for rec in all_potential_recommendations:
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    ground_truth.add((n1,n2,0))


## ALS (Alternating Least Squares)

In [74]:
#Definition that converts the users id to user names
def number_to_username(alg_list: list, g) -> list:
    alg_list_transformed=[]
    for i in range(len(alg_list)):
        name=g.vs[int(alg_list[i][0])]['name']
        new_tuple=(name, alg_list[i][1])
        alg_list_transformed.append(new_tuple)
    return alg_list_transformed

#It returns the user recommendations given a user_id
def recommend_users(user_id: int, G:csr_matrix, g: igraph.Graph, top: int = 10) -> list:
    ALS_recommended_users=model.recommend(user_id, G, top)
    return ALS_recommended_users

In [75]:
#Create an adjency matrix from the graph
G = graph.get_adjacency().data
#Convert the adjency matrix to csr_matrix, which is the variable type needed for doing ALS 
G = csr_matrix(G)

In [76]:
#Initialize ALS model
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=5, calculate_training_loss=True)
#Train ALS model
model.fit(G)

In [77]:
#Get user ids recommendations for a user
ALS_recomdetaion_ids=recommend_users(user_id, G, graph)
#Transform the user ids to their repective name
ALS_recomdetaion_names=number_to_username(ALS_recomdetaion_ids, graph)
print(f"Recomendations for user {user_name} with id {user_id} using ALS (Alternate Least Squares):")
ALS_recomdetaion_names

Recomendations for user Liensevi with id 0 using ALS (Alternate Least Squares):


[('LouDobbs', 1.27170915e-05),
 ('GenFlynn', 1.1479422e-05),
 ('drdavidsamadi', 7.779039e-06),
 ('KimM53904472', 3.3860933e-06),
 ('thewatchfulmom', 3.2320272e-06),
 ('aarfreethinker', 3.2208013e-06),
 ('michellemalkin', 3.0228725e-06),
 ('RyanAFournier', 2.9403664e-06),
 ('RebekahKirkla15', 2.8612235e-06),
 ('paulsperry_', 2.7980038e-06)]

## ALS Testing

In [78]:
## ALS testing

In [79]:
def predict_ALS(testset, model):
    """
    predict for a list of observations the score for adding/removing a link
    """
    # initialize the empty list
    all_predictions = []
    # scroll the obs
    for n1,n2, _ in testset:
        # take here the low-dimensional vectors returned by the matrix factorization
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]
        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)
        all_predictions.append(one_p)
        
    return all_predictions

In [80]:
# generate the predictions
df_test_als = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])
all_predictions = predict_ALS(df_test_als.values, model)
# add predictions to df
df_test_als["rating"] = all_predictions
# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test_als["rating"] = df_test_als["rating"].apply(lambda x: round(x))

In [81]:
# number of observations matched by the prediction
right_predictions_als = len(df_test_als[df_test_als.edge == df_test_als.rating])
# accuracy
print(f"Accuracy for ALS: {right_predictions_als/len(df_test_als)}")

Accuracy for ALS: 0.9299307958477508


 ----------------------------------------------------------------------------------------------------------------------------

## Adamic-Adar

In [82]:
def get_neighbors(user_a, g: igraph.Graph) -> list:
    #We only need to consider those verices at distance 2
    #We take those users at distance 1
    neighbors_1=set(g.neighborhood(user_a, order=1))
    #We take those users at distance 1 & 2
    neighbors_2=set(g.neighborhood(user_a, order=2))
    #We take only those nodes that are at distance 2
    return list(neighbors_2 - neighbors_1)
#Implementation of Adamic-Adar algorithm

def get_recommendation_AA(username: int, g:igraph.Graph) -> pd.DataFrame:
    neighbors_only_order_2=get_neighbors(username, g)
    #Initialize dataframe with the user we want to recommend to as the column, and their 2-distance neighbors as indexes
    adamic_adar_data=pd.DataFrame(columns=[username], index=neighbors_only_order_2)
    #For every 2-distance users compute AA(x,y)
    for user_y in neighbors_only_order_2:
        if username!=user_y:
            #Get neighbors for the two nodes
            x_neighbors=set(g.neighborhood(username))
            y_neighbors=set(g.neighborhood(user_y))
        #Get only those nodes that are neighbors of both nodes
        same_neighbors=x_neighbors&y_neighbors
        aa_val=0
        #Compute the Adamic-Avar value and add it to the dataframe
        for n in same_neighbors:
            num_neighbors=len(g.neighbors(n))
            aa_val+=(1/math.log(num_neighbors,10))
        adamic_adar_data[username][user_y]=aa_val
    #Sort values and return the top 10 recommendations
    top_n_recommendations_aa=adamic_adar_data[username].sort_values(ascending=False)
    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
    return aa_final_recommendation

def pair_recommendation_AA(user_a: int, user_b: int, graph: igraph.Graph) -> int:
    a_neighbors=set(graph.neighborhood(user_a))
    b_neighbors=set(graph.neighborhood(user_b))
    common_neighbors=a_neighbors&b_neighbors
    aa_val=0
    #Compute the Adamic-Avar value and return
    for n in common_neighbors:
        if user_a!=n:
            num_neighbors=len(graph.neighbors(n))
            aa_val+=(1/math.log(num_neighbors,10))
    return aa_val

#Transform user ids to user names
def AA_num_to_name(dataset: pd.DataFrame, g: igraph.Graph)->pd.DataFrame:
    old_indices=list(dataset.index)
    new_indices=[]
    main_user_id=dataset.columns[0]
    main_user_name=g.vs[main_user_id]['name']
    for user_id in old_indices:
        name=g.vs[user_id]['name']
        new_indices.append(name)
    new_dataset=pd.DataFrame(dataset.values, columns=[main_user_name], index=new_indices)
    return new_dataset

In [83]:
#Get recommended users ids for the requested user 
AA_recommendation_ids=get_recommendation_AA(user_id, graph)
#Transform the ids to usernames
AA_recommendation_names=AA_num_to_name(AA_recommendation_ids, graph)
print(f"Recomendations for user {user_name} with id {user_id} using Adamic-Adar:")
AA_recommendation_names

Recomendations for user Liensevi with id 0 using Adamic-Adar:


,Liensevi
N30Foll0w,1.43068
FloydStad,1.43068
kasmouse,1.43068
dennis91842840,1.43068


## Adamic Adar Testing

In [84]:
def predict_AA(testset, graph):
    """
    predict for a list of observations the score for adding/removing a link
    """
    # initialize the empty list
    all_predictions = []
    # scroll the obs
    for n1,n2, _ in testset:
        # take here the low-dimensional vectors returned by the matrix factorization
        if n1!=n2:
            try:
                aa_score = pair_recommendation_AA(n1,n2, graph)
            except:
                aa_score=0
        all_predictions.append(aa_score)
        
    return all_predictions

In [85]:
# generate the predictions
df_test_aa = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])
all_predictions_aa = predict_AA(df_test_aa.values, graph)
# add predictions to df
df_test_aa["rating"] = all_predictions_aa
# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test_aa["rating"] = df_test_aa["rating"].apply(lambda x: 1 if x>=1 else 0)

In [86]:
# number of observations matched by the prediction
right_predictions_aa = len(df_test_aa[df_test_aa.edge == df_test_aa.rating])
# accuracy
print(f"Accuracy for Adamic-Adar: {right_predictions_aa/len(df_test_aa)}")

Accuracy for Adamic-Adar: 0.8515354671280276


----------------------------------------------------------------------------------------------------------------------------

## PageRank

In [87]:
#Transform user ids to user names
def pagerank_clearer(pagerank_values: list, g: igraph.Graph) -> list:
    pagerank=[]
    for i in range(len(pagerank_values)):
            user=pagerank_values[i][0]
            name=g.vs[user]['name']
            val=float(pagerank_values[i][1])
            pagerank.append((name, val))
    return pagerank

#From the score obtained from PageRank algorithm, get top user ids with higher score and that are 2-distance neighbors
def top_10_ids(pagerank_result: list, user_id: int, graph: igraph.Graph)->list:
    pagerank_with_ids=[]
    
    #We only need to consider those verices at distance 2
    #We take those users at distance 1
    neighbors_1=set(graph.neighborhood(user_id, order=1))
    #We take those users at distance 1 & 2
    neighbors_2=set(graph.neighborhood(user_id, order=2))
    #We take only those nodes that are at distance 2
    neighbors_only_order_2=list(neighbors_2 - neighbors_1)
    
    for i in range(len(pagerank_result)):
        if i!=user_id and i in neighbors_only_order_2:
            pagerank_with_ids.append([i, pagerank_result[i]])
    pagerank_with_ids.sort(key = lambda x: x[1], reverse=True)
    pagerank_top_10_ids=pagerank_with_ids[0:10]
    return pagerank_top_10_ids

In [88]:
#Get PageRank scores for each user given the initial node/user
PageRank_recommendations=graph.personalized_pagerank(reset_vertices=user_id)
#Get top users ids with higher score and that are 2-distance neighbor 
PageRank_recommendation_ids=top_10_ids(PageRank_recommendations, user_id, graph)
#Transform the ids to usernames
PageRank_recommendation_names=pagerank_clearer(PageRank_recommendation_ids, graph)
print(f"Recomendations for user {user_name} with id {user_id} using Personalized PageRank:")
PageRank_recommendation_names

Recomendations for user Liensevi with id 0 using Personalized PageRank:


[('kasmouse', 0.0807136386394075),
 ('dennis91842840', 0.06518142272913913),
 ('N30Foll0w', 0.06518142272913913),
 ('FloydStad', 0.06518142272913913)]

## Personalized Pagerank testing

In [89]:
def predict_pr(testset, graph):
    all_predictions = []
    for n1, n2, _ in testset:
        ranking = graph.personalized_pagerank(vertices=n2, reset_vertices = n1)
        all_predictions.append(ranking)
    return all_predictions

In [90]:
# generate the predictions
df_test_pr = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])
all_predictions_pr = predict_pr(df_test_pr.values,graph)
# add predictions to df
df_test_pr["rating"] = all_predictions_pr
# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test_pr["rating"] = df_test_pr["rating"].apply(lambda x: 1 if x > 0.05 else 0)

In [91]:
# number of observations matched by the prediction
right_predictions_pr = len(df_test_pr[df_test_pr.edge == df_test_pr.rating])
# accuracy
print(f"Accuracy for Personalized Pagerank: {right_predictions_pr/len(df_test_pr)}")

Accuracy for Personalized Pagerank: 0.9359320934256056


----------------------------------------------------------------------------------------------------------------------------

## Node2vec

In [14]:
#In order to apply Node2vec, we need to convert the igraph to a networkx graph
A = graph.get_edgelist()
nx_graph = nx.Graph(A) # In case your graph is directed

In [15]:
#Initialize Node2vec model
n2v = node2vec.Node2Vec(nx_graph, dimensions=64, walk_length=2, num_walks=200, workers=4) 
#Train Node2vec model
model = n2v.fit(window=10, min_count=1, batch_words=4)

In [16]:
#Get a list of the recommended users with their respective scores of the initial/main node
user_id_str=str(user_id)
node2vec_recommendation_ids=model.wv.most_similar(user_id_str)

In [17]:
node2vec_recommendation_ids

[('1286', 0.9995108246803284),
 ('3246', 0.9993603229522705),
 ('272', 0.9992141723632812),
 ('3666', 0.9215577840805054),
 ('2878', 0.7088077664375305),
 ('9946', 0.7081505060195923),
 ('11764', 0.7076790928840637),
 ('9936', 0.7061811685562134),
 ('1800', 0.7060530781745911),
 ('13430', 0.7050497531890869)]

In [96]:
#Transform user ids to user names
node2vec_recommendation_names=number_to_username(node2vec_recommendation_ids, graph)

In [97]:
print(f"Recomendations for user {user_name} with id {user_id} using Node2vec:")
node2vec_recommendation_names

Recomendations for user Liensevi with id 0 using Node2vec:


[('FloydStad', 0.9991071224212646),
 ('N30Foll0w', 0.9989970922470093),
 ('dennis91842840', 0.9987884759902954),
 ('kasmouse', 0.9028507471084595),
 ('Nic04588534', 0.6528711318969727),
 ('IvonneMeeuwsen', 0.651504635810852),
 ('RussLSmith', 0.6512831449508667),
 ('margie_wateland', 0.6504145860671997),
 ('DocLionel1', 0.6494978666305542),
 ('JimWatkins', 0.648861825466156)]

## Node2Vec Testing

In [98]:
def predict_nv(testset, model):
    all_predictions = []
    for n1, n2, _ in testset:
        ranking = model.wv.similarity(str(n1), str(n2))
        all_predictions.append(ranking)
    return all_predictions

In [99]:
# generate the predictions
df_test_nv = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])
all_predictions_nv = predict_nv(df_test_nv.values, model)
# add predictions to df
df_test_nv["rating"] = all_predictions_nv
# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test_nv["rating"] = df_test_nv["rating"].apply(lambda x: round(x))

In [100]:
# number of observations matched by the prediction
right_predictions_nv = len(df_test_nv[df_test_nv.edge == df_test_nv.rating])
# accuracy
print(f"Accuracy for Node2Vec: {right_predictions_nv/len(df_test_nv)}")

Accuracy for Node2Vec: 0.027411332179930796


----------------------------------------------------------------------------------------------------------------------------

## TF-IDF + Cosine Similarity Recommender System

As we have seen in the theories, TF-IDF + Cosine Similarity or Pearson Coefficient can also recommend potential items to a given user. In this case, we did something similar, but this time, given a tweet it recommends other tweets. The thing to demonstrate here is that a recommender system is almost the same as a search engine, but now the "query" is the user profile, or in our case a tweet.

In [5]:
def tf_idf_custom_score(user: str, tweets: pd.DataFrame):
    user_sample_tweet=''
    # Relevant document scoring
    for i in range(len(tweets)):
        if tweets['user'][i]['screen_name'] == user:
            user_sample_tweet = tweets['text'][i]
            break
    if user_sample_tweet == '':
        return
    tweets_score = search_engine.ranking_system_ex_3.cosine_similarity(
        user_sample_tweet, list(tweets.index)
    )
    results = pd.DataFrame(columns = ["User", "Similarity"])
    # Score assignation
    recommended_tweets = []
    cosine_similarities = []
    for i in range(len(tweets)):
        recommended_tweets.append(tweets["user"][i]['screen_name'])
        cosine_similarities.append(tweets_score[i])

    results["Similarity"] = cosine_similarities
    results["User"] = recommended_tweets

    # Sort by descending score
    results_sorted = results.sort_values(
        by=["Similarity"], ascending=False
    )

    return results_sorted.reset_index(drop=True)
    

In [6]:
test = tf_idf_custom_score(user_name, search_engine.tweets)

In [7]:
test.head(10)

,User,Similarity
0,Liensevi,1.000000
1,N30Foll0w,1.000000
2,FloydStad,1.000000
3,lah3309,0.235331
4,LivingOhioDream,0.217845
5,Ace77ofnocal,0.214249
6,DebraH74710152,0.176193
7,cockyman7,0.176193
8,stofer99,0.176193
9,jcounselman3,0.176193
